In [33]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
df_train = pd.read_csv('../data/processed/train_processed.csv')
df_test = pd.read_csv ('../data/processed/test_processed.csv')

In [35]:
params_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

In [36]:
X = df_train.drop(['Survived', 'PassengerId'], axis=1)
y = df_train['Survived'].copy()
pass_ids_df_test = df_test['PassengerId'].copy()
X_test_final = df_test.drop(['PassengerId'], axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
model = RandomForestClassifier(random_state=42)
#model.fit(X_train, y_train)

grid_search = GridSearchCV(estimator=model, param_grid=params_grid,cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


print("\nMejores parámetros encontrados por GridSearchCV:")
print(grid_search.best_params_)

print(f"Mejor Accuracy en CV: {grid_search.best_score_:.4f}")

# Obtener el mejor modelo entrenado
best_model = grid_search.best_estimator_
print("\nEl mejor modelo ya ha sido re-entrenado con los mejores parámetros en todo X_train.")

# Ahora, evalúa este best_model en el conjunto de validación (X_val, y_val)
print("\nEvaluando el MEJOR modelo en el conjunto de validación (X_val):")
y_pred_best = best_model.predict(X_val)
accuracy_best = accuracy_score(y_val, y_pred_best)
print(f"Accuracy del MEJOR modelo en X_val: {accuracy_best:.4f}")

# Opcional: Mostrar Matriz de Confusión y Reporte de Clasificación para el mejor modelo
print("\nMatriz de Confusión del MEJOR Modelo:")
print(confusion_matrix(y_val, y_pred_best))

print("\nReporte de Clasificación del MEJOR Modelo:")
print(classification_report(y_val, y_pred_best, target_names=['No Sobrevive (0)', 'Sobrevive (1)']))

Fitting 5 folds for each of 162 candidates, totalling 810 fits

Mejores parámetros encontrados por GridSearchCV:
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Mejor Accuracy en CV: 0.8357

El mejor modelo ya ha sido re-entrenado con los mejores parámetros en todo X_train.

Evaluando el MEJOR modelo en el conjunto de validación (X_val):
Accuracy del MEJOR modelo en X_val: 0.8101

Matriz de Confusión del MEJOR Modelo:
[[94 16]
 [18 51]]

Reporte de Clasificación del MEJOR Modelo:
                  precision    recall  f1-score   support

No Sobrevive (0)       0.84      0.85      0.85       110
   Sobrevive (1)       0.76      0.74      0.75        69

        accuracy                           0.81       179
       macro avg       0.80      0.80      0.80       179
    weighted avg       0.81      0.81      0.81       179

